<a href="https://colab.research.google.com/github/younglaecho/learn_machine_learning/blob/main/chap05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

## 랜덤 포레스트
결정트리를 램덤하게 만들어 결정 트리의 숲을 만드는 알고리즘
* 부트스트랩 샘플 
  - 중복을 허용하는 샘플
  - 일반적으로, 샘플의 데이터 개수는 훈련 세트의 크기와 같게 만듦

* RandomForestClassifier
  - 기본적으로 노드마다 전체 특성 개수의 제곱급만큼의 특성을 선택합니다.
  - 최선의 분할을 찾음.

####데이터 전처리

In [9]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

cross_validate의 return_train_score=True 로 지정해야 train_score를 확인할 수 있습니다.


In [7]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


rf.feature_importances_를 통해서 특성의 중요도

In [16]:
rf.fit(train_input, train_target)
print(rf.score(train_input,train_target))
print(rf.score(test_input,test_target))
print(rf.feature_importances_) # 훈련 세트에 훈련한 후 특성 중요도

0.996921300750433
0.8892307692307693
[0.23167441 0.50039841 0.26792718]


oob_score=True : 부트스트랩 샘플에 포함되지 않은 샘플을 검증세트에 사용함

In [19]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


##엑스트라 트리

결정트리를 램덤하게 만들어 결정 트리의 숲을 만드는 알고리즘

* 부트스트랩 샘플을 사용하지 않음 => 중복 허용x, 전체 훈련 세트를 사용
* 기본적으로 노드마다 전체 특성 개수의 제곱급만큼의 특성을 선택합니다.
* 노드를 분할할 때 무작위로 분할

In [20]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
stores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


* 더 높은 무작위성 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 합니다.
* 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점입니다.

In [21]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그래디언트 부스팅

* 기본적으로 깊이가 3인 결정 트리를 100개 사용함.
* 깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강함.
* 경사하강법을 사용하여 트리를 앙상블에 추가함.
* 분류 : 로지스틱 손실 함수
* 회귀 : 평균 제곱 오차 함수

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


GradientBoostingClassifier의 n_estimaors(결정 트리의 개수), learning_rate(학습율)을 조정할 수 있습니다.


In [24]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [25]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


subsample : 트리 훈련에 사용할 훈련 세트의 비율을 정함
  - 1보다 작으면 훈련 세트의 일부만 사용합니다.
  - 확률적 경사하강법이나, 미니배치 경사 하강법과 비슷함.

일반적으로 그래디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있습니다.<br>
순서대로 트리를 추가하기 때문에 훈련 속도가 느립니다.

##히스토그램 기반 그래디언트 부스팅


입력 특성을 256개의 구간으로 나눔. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음.<br>
256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용합니다.


In [27]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


n_estimators 대신에 부스팅 반복 횟수를 max_iter를 사용합니다.<Br>
permutation_importance() 함수를 사용해서 부스팅의 특성 중요도를 계산해보겠습니다.<br>
n_repeats 매개변수는 랜덤하게 섞을 횟수를 지접합니다. 

In [28]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [29]:
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [30]:
hgb.score(test_input, test_target)

0.8723076923076923